# Kmean clustering by topics weights to find similar restaurants

## Import necessary libraries 

In [107]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import math

## Read the restaurant business with topics weights datasets

In [108]:
pos_restaurant_topic_file = '../Datasets/Positive_Restaurant_Topics.csv'
df_pos_restaurant_topic = pd.read_csv(pos_restaurant_topic_file)
del df_pos_restaurant_topic['Unnamed: 0']

In [109]:
df_pos_restaurant_topic.head(3)

,business_id,name,categories,city,stars,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6
0,-2-ih3mE8KPyeKVIzpBfPQ,SkyGarten,"American (Traditional), Bars, Restaurants, Nig...",Philadelphia,3.5,0.170043,0.000000,0.00000,0.014064,0.145168,0.670726
1,-T_lkOvaK39R-Ufg6VUyxg,Magpie,"Desserts, Food, Coffee & Tea, Restaurants",Philadelphia,4.5,0.499080,0.016986,0.18418,0.145433,0.098592,0.055729
2,-V0vIgo6196MDn_x3ZaYmA,La Creperie Cafe,"Restaurants, Cafes, Creperies",Philadelphia,3.5,0.224335,0.164131,0.03775,0.227311,0.272450,0.074022


In [110]:
df_pos_clus_present = df_pos_restaurant_topic[['business_id' , 'name' , 'topic_1' , 'topic_2' , 'topic_3' , 'topic_4' , 'topic_5' , 'topic_6']]
df_pos_clus_present.head(3)

,business_id,name,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6
0,-2-ih3mE8KPyeKVIzpBfPQ,SkyGarten,0.170043,0.000000,0.00000,0.014064,0.145168,0.670726
1,-T_lkOvaK39R-Ufg6VUyxg,Magpie,0.499080,0.016986,0.18418,0.145433,0.098592,0.055729
2,-V0vIgo6196MDn_x3ZaYmA,La Creperie Cafe,0.224335,0.164131,0.03775,0.227311,0.272450,0.074022


In [111]:
df_pos_res_clus =  df_pos_restaurant_topic[['topic_1' , 'topic_2' , 'topic_3' , 'topic_4' , 'topic_5' , 'topic_6']]
df_pos_res_clus.head(3)

,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6
0,0.170043,0.000000,0.00000,0.014064,0.145168,0.670726
1,0.499080,0.016986,0.18418,0.145433,0.098592,0.055729
2,0.224335,0.164131,0.03775,0.227311,0.272450,0.074022


## Apply K-Means clustering method

In [112]:
kmeans = KMeans(n_clusters=6).fit(df_pos_res_clus) #compute k-means clustering
centroids = kmeans.cluster_centers_ # get cluster center
clusters = kmeans.predict(df_pos_res_clus)
df_pos_restaurant_topic.insert(11,"cluster",clusters,True)
print(centroids)


[[0.25790211 0.07230473 0.0314339  0.37047974 0.18766266 0.08021685]
 [0.55596943 0.04925348 0.02603008 0.12916122 0.1769055  0.06268029]
 [0.11747272 0.04679387 0.61751602 0.06879234 0.09546997 0.05395508]
 [0.18804789 0.46598931 0.0430542  0.13922931 0.1103424  0.05333689]
 [0.16490808 0.05234165 0.02587004 0.13162187 0.18819902 0.43705933]
 [0.3163212  0.05819385 0.02531286 0.10615383 0.40370722 0.09031103]]


In [113]:
df_pos_restaurant_topic.head(3)

,business_id,name,categories,city,stars,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,cluster
0,-2-ih3mE8KPyeKVIzpBfPQ,SkyGarten,"American (Traditional), Bars, Restaurants, Nig...",Philadelphia,3.5,0.170043,0.000000,0.00000,0.014064,0.145168,0.670726,4
1,-T_lkOvaK39R-Ufg6VUyxg,Magpie,"Desserts, Food, Coffee & Tea, Restaurants",Philadelphia,4.5,0.499080,0.016986,0.18418,0.145433,0.098592,0.055729,1
2,-V0vIgo6196MDn_x3ZaYmA,La Creperie Cafe,"Restaurants, Cafes, Creperies",Philadelphia,3.5,0.224335,0.164131,0.03775,0.227311,0.272450,0.074022,0


In [114]:
df_pos_restaurant_topic_c0 = df_pos_restaurant_topic[df_pos_restaurant_topic['cluster'] == 0]
df_pos_restaurant_topic_c1 = df_pos_restaurant_topic[df_pos_restaurant_topic['cluster'] == 1]
df_pos_restaurant_topic_c2 = df_pos_restaurant_topic[df_pos_restaurant_topic['cluster'] == 2]
df_pos_restaurant_topic_c3 = df_pos_restaurant_topic[df_pos_restaurant_topic['cluster'] == 3]
df_pos_restaurant_topic_c4 = df_pos_restaurant_topic[df_pos_restaurant_topic['cluster'] == 4]
df_pos_restaurant_topic_c5 = df_pos_restaurant_topic[df_pos_restaurant_topic['cluster'] == 5]

## Restaurant similar to 'Horizons'

Function : retrieve_similar_res
</br>
Parameter : restaurant name , restaurant number
</br>
output : a list of similar restaurants

In [118]:
def find_closest_distance(res_name):
    res_info = df_pos_restaurant_topic[df_pos_restaurant_topic['name'] == res_name]
    res_info = res_info.reset_index(drop=True)
    res_cluster = res_info['cluster'][0]
    df_result = df_pos_restaurant_topic[(df_pos_restaurant_topic['cluster'] == res_cluster) & (df_pos_restaurant_topic['name'] != res_name)]  
    df_result = df_result.reset_index(drop=True)
    df_result.insert(0 , 'distance' , -1.0 , True)
    
    res_pos = [ res_info['topic_1'][0] , res_info['topic_2'][0] , res_info['topic_3'][0] , res_info['topic_4'][0] , res_info['topic_5'][0] , res_info['topic_6'][0]]
    for i in range(0 , len(df_result)):
        curr_res = df_result.loc[i]
        curr_res_pos = [ curr_res['topic_1'] , curr_res['topic_2'] , curr_res['topic_3'] , curr_res['topic_4'] , curr_res['topic_5'] , curr_res['topic_6']]
        df_result.loc[i,'distance'] = math.dist(res_pos , curr_res_pos)
        df_result['distance'] = pd.to_numeric(df_result['distance'])
        df_result = df_result.sort_values(by=['distance'] , ascending=True)
    return df_result
    

In [123]:
def retrieve_similar_res(res_name , res_num):
    res_info = df_pos_restaurant_topic[df_pos_restaurant_topic['name'] == res_name]
    res_info = res_info.reset_index(drop=True)
    res_cluster = res_info['cluster'][0]
    #df_result = df_pos_restaurant_topic[(df_pos_restaurant_topic['cluster'] == res_cluster) & (df_pos_restaurant_topic['name'] != res_name)]
    #df_result = df_result.reset_index(drop=True)
    df_result = find_closest_distance(res_name)
    print("Restaurant Similar to '{}': ".format(res_name))
    print('='*50)
    for n in range(0 , res_num):
        print('Restaurant: {} \ncategories: {:50}'.format(df_result['name'][n], df_result['categories'][n]))
        print('-'*50)


In [124]:
retrieve_similar_res("Horizons" , 5)

Restaurant Similar to 'Horizons': 
Restaurant: Magpie 
categories: Desserts, Food, Coffee & Tea, Restaurants         
--------------------------------------------------
Restaurant: Le Chéri 
categories: Restaurants, French                               
--------------------------------------------------
Restaurant: DaMò Pasta Lab 
categories: Italian, Pasta Shops, Restaurants, Food, Specialty Food
--------------------------------------------------
Restaurant: Oishii Poké 
categories: Restaurants, Mexican, Food, Hawaiian, Poke, Japanese
--------------------------------------------------
Restaurant: Il Pittore 
categories: Restaurants, American (New), Diners, Italian      
--------------------------------------------------
